In [89]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [90]:
#Load the data into a pandas dataframe
df = pd.read_csv("data/TorontoListings.csv")
df

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1419,https://www.airbnb.com/rooms/1419,20221206172219,2022-12-07,previous scrape,Beautiful home in amazing area!,"This large, family home is located in one of T...",The apartment is located in the Ossington stri...,https://a0.muscache.com/pictures/76206750/d643...,1565,...,5.00,5.00,5.00,NaN,f,1,1,0,0,0.08
1,8077,https://www.airbnb.com/rooms/8077,20221206172219,2022-12-06,city scrape,Downtown Harbourfront Private Room,Guest room in a luxury condo with access to al...,NaN,https://a0.muscache.com/pictures/11780344/141c...,22795,...,4.90,4.92,4.83,NaN,t,2,1,1,0,1.04
2,26654,https://www.airbnb.com/rooms/26654,20221206172219,2022-12-07,city scrape,World Class downtown @CN Tower Theatre MTCC ga...,"CN Tower, TIFF Bell Lightbox, Metro Convention...",There's a reason they call it the Entertainmen...,https://a0.muscache.com/pictures/81811785/5dcd...,113345,...,4.75,4.85,4.65,NaN,f,3,3,0,0,0.28
3,624190,https://www.airbnb.com/rooms/624190,20221206172219,2022-12-07,previous scrape,Danforth/Greek Town appartment,<b>The space</b><br />One bedroom with queen s...,NaN,https://a0.muscache.com/pictures/7942482/19176...,3096162,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
4,625877,https://www.airbnb.com/rooms/625877,20221206172219,2022-12-07,city scrape,Calming Bedroom 2 - Beaches Toronto,<b>The space</b><br />We have three well appoi...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,2268652,...,4.95,4.84,4.95,NaN,f,3,0,3,0,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16756,774857571381461391,https://www.airbnb.com/rooms/774857571381461391,20221206172219,2022-12-07,city scrape,Entire Condo Right Downtown With Great Views,** High Speed Internet (500Mbps+) for those wh...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,277111026,...,NaN,NaN,NaN,STR-2105-FSRXVC,t,3,3,0,0,NaN
16757,774859597284156992,https://www.airbnb.com/rooms/774859597284156992,20221206172219,2022-12-07,city scrape,Perfect DT Loft w/ View for Work or Adventure,** Guests can quarantine here<br />** High Spe...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,277111026,...,NaN,NaN,NaN,STR-2105-FSRXVC,t,3,3,0,0,NaN
16758,774864431762589684,https://www.airbnb.com/rooms/774864431762589684,20221206172219,2022-12-06,city scrape,Chic Midtown Toronto condo,Enjoy this modern one bedroom space with a be...,"You can find many restaurants, bars, shops and...",https://a0.muscache.com/pictures/miso/Hosting-...,42048792,...,NaN,NaN,NaN,STR-2211-JFHBHZ,t,4,4,0,0,NaN
16759,774877214800883103,https://www.airbnb.com/rooms/774877214800883103,20221206172219,2022-12-07,city scrape,"Urban Oasis, Panoramic CN Tower View, FREE Par...",In a charming neighborhood with wonderful rest...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,474405217,...,NaN,NaN,NaN,STR-2207-FJJBHB,t,2,1,1,0,NaN


In [91]:
#Drop columns not needed for analysis
keep_cols = ["host_location", "host_is_superhost", "host_neighbourhood", "host_identity_verified", "neighbourhood_cleansed", "latitude", "longitude", "property_type", "room_type", "accommodates", "bathrooms_text", "bedrooms", "beds", "amenities", "price", "minimum_nights", "maximum_nights", "has_availability", "availability_365", "number_of_reviews", "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value", "license", "calculated_host_listings_count", "reviews_per_month"]
df = df.loc[:, keep_cols]
df.columns

Index(['host_location', 'host_is_superhost', 'host_neighbourhood',
       'host_identity_verified', 'neighbourhood_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_365', 'number_of_reviews', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'license',
       'calculated_host_listings_count', 'reviews_per_month'],
      dtype='object')

In [92]:
# Convert bathroom_text column into numeric value, shared baths count as 0.5x a private bath
df["bathrooms"] = df["bathrooms_text"].str.replace(' shared ', '.5').str.extract(r'(\d+\.?\d*)').astype(float)
df.drop("bathrooms_text", axis=1, inplace=True)

In [93]:
# Convert price text column into numerical value
df["price"] = df["price"].apply(lambda x: float(x.replace("$", "").replace(",", "")))

In [94]:
# Apply KNNImputer to dataframe subset
cols_to_impute = ["accommodates", "bedrooms", "beds", "bathrooms", "price", "latitude", "longitude"]
df_to_impute = df[cols_to_impute]
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df_to_impute), columns=df_to_impute.columns)

# Replace the missing values in original df with imputed values
df[cols_to_impute] = df_imputed

In [95]:
# Remove listings with missing values in important columns
important_cols = ["neighbourhood_cleansed", "property_type", "room_type", "accommodates", "bedrooms", "beds", "price", "latitude", "longitude"]
df = df.dropna(subset = important_cols)

In [96]:
df.columns

Index(['host_location', 'host_is_superhost', 'host_neighbourhood',
       'host_identity_verified', 'neighbourhood_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bedrooms',
       'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'has_availability', 'availability_365', 'number_of_reviews',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'reviews_per_month', 'bathrooms'],
      dtype='object')

In [97]:
# Use one-hot encoding to convert categorical columns into numerical values
categorical_cols = ["neighbourhood_cleansed", "property_type", "room_type", "host_location", "host_neighbourhood",]
df = pd.get_dummies(df, columns = categorical_cols)

In [98]:
# Replace 't' and 'f' with 1 and 0, respectively
df[["host_is_superhost", "host_identity_verified", "has_availability"]] = df[["host_is_superhost", "host_identity_verified", "has_availability"]].replace({"t": 1, "f": 0})

In [ ]:
# Change License column to 1 if a license exists and 0 if a license doesn't exist
df["license"] = df["license"].fillna(0)
df.loc[df["license"] != 0, "license"] = 1

In [99]:
# Normalize numerical columns
# numerical_cols = ["price", "minimum_nights", "maximum_nights", "calculated_host_listings_count", "accommodates", "bedrooms", "beds", "bathrooms", "number_of_reviews", "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", "review_scores_checkin", "review_scores_communication", "review_scores_location", "reviews_per_month"]
# df[numerical_cols] = (df[numerical_cols] - df[numerical_cols].mean()) / df[numerical_cols].std()

In [100]:
# Create columns for each amenity with values 0 or 1 indicating whether an amenity is present or not
amenities_df = df["amenities"].str.get_dummies(sep=',')

# join amenities dataframe to original dataframe
df = df.join(amenities_df)
df = df.drop("amenities", axis = 1)

In [101]:
# Remove outliers
df = df[df["price"] <= 700]

In [102]:
df.to_csv('data/cleaned_toronto_dataset.csv', index=False)